In [1]:
from __future__ import division, print_function, unicode_literals
import numpy as np

# LSTM Reference implementation in Numpy

This implementation is meant as a reference for understanding and to check other implementations. 
The figures and formulas are taken from ["LSTM: A Search Space Odyssey"](http://arxiv.org/abs/1503.04069).
It is not optimized for speed or memory consumption in any way.

We use the following activation functions (all pointwise on vector inputs):

  * The logistic sigmoid for all the gates: $\sigma(\mathbf{x}) = \frac{1}{1+e^{-\mathbf{x}}}$ 
  * hyperbolic tangent for block input and output: $g(\mathbf{x}) = h(\mathbf{x}) = \text{tanh}(\mathbf{x}) = \frac{e^\mathbf{x} - e^{-\mathbf{x}}}{e^\mathbf{x} + e^{-\mathbf{x}}}$
  
with the corresponing derivatives:
  * $\sigma'(\mathbf{x}) = \sigma(\mathbf{x}) \odot (\mathbf{1} - \sigma(\mathbf{x})) $
  * $\text{tanh}'(\mathbf{x}) = \mathbf{1} - \text{tanh}(\mathbf{x})^2$

In [2]:
sigma = lambda x: 1./(1 + np.exp(-x))
sigma_deriv = lambda x: sigma(x) * (1 - sigma(x))

g = lambda x: np.tanh(x)
g_deriv = lambda x: 1 - g(x)**2

h = lambda x: np.tanh(x)
h_deriv = lambda x: 1 - h(x)**2

## Weights
Let $N$ be the number of LSTM blocks and $M$ the number of inputs. Then we get the following weights:

 * Input weights: $\mathbf{W}_z$, $\mathbf{W}_s$, $\mathbf{W}_f$, $\mathbf{W}_o$ $\in \mathbb{R}^{M \times N}$
 * Recurrent weights: $\mathbf{R}_z$, $\mathbf{R}_s$, $\mathbf{R}_f$, $\mathbf{R}_o$ $\in \mathbb{R}^{N \times N}$
 * Peephole weights: $\mathbf{p}_s$, $\mathbf{p}_f$, $\mathbf{p}_o$ $\in \mathbb{R}^{N}$
 * Bias weights: $\mathbf{b}_z$, $\mathbf{b}_s$, $\mathbf{b}_f$, $\mathbf{b}_o$ $\in \mathbb{R}^{N}$

Where the subscript letters stand for the block input ($z$), the input gate ($i$), the forget gate ($f$), and the output gate($o$). 

In [3]:
M = 2  # dimensionality of inputs
N = 3  # number of LSTM blocks

rnd = np.random.RandomState()

Wz = rnd.randn(M, N) * 0.1
Wi = rnd.randn(M, N) * 0.1
Wf = rnd.randn(M, N) * 0.1
Wo = rnd.randn(M, N) * 0.1

Rz = rnd.randn(N, N) * 0.1
Ri = rnd.randn(N, N) * 0.1
Rf = rnd.randn(N, N) * 0.1
Ro = rnd.randn(N, N) * 0.1

pi = rnd.randn(N) * 0.1
pf = rnd.randn(N) * 0.1
po = rnd.randn(N) * 0.1

bz = rnd.randn(N) * 0.1
bi = rnd.randn(N) * 0.1
bf = rnd.randn(N) * 0.1
bo = rnd.randn(N) * 0.1

weights = (Wz, Wi, Wf, Wo, Rz, Ri, Rf, Ro, pi, pf, po, bz, bi, bf, bo)

## Forward pass

<img src="http://people.idsia.ch/~greff/lstm.svg" alt="LSTM" style="float:right;" />

Formulas:

$\bar{\mathbf{z}}^t = \mathbf{x}^t \mathbf{W}_z + \mathbf{y}^{t-1} \mathbf{R}_z + \mathbf{b}_z $<br>
$\mathbf{z}^t = g(\bar{\mathbf{z}}^t) $
*(block input)*

$\bar{\mathbf{i}}^t = \mathbf{x}^t \mathbf{W}_i + \mathbf{y}^{t-1} \mathbf{R}_i + \mathbf{p}_i \odot \mathbf{c}^{t-1} + \mathbf{b}_i $<br>
$\mathbf{i}^t = \sigma(\bar{\mathbf{i}}^t) $
*(input gate)*
  
$\bar{\mathbf{f}}^t = \mathbf{x}^t \mathbf{W}_f + \mathbf{y}^{t-1} \mathbf{R}_f + \mathbf{p}_f \odot \mathbf{c}^{t-1} + \mathbf{b}_f $<br>
$\mathbf{f}^t = \sigma(\bar{\mathbf{f}}^t) $
*(forget gate)*
  
$\mathbf{c}^t = \mathbf{z}^t \odot \mathbf{i}^t + \mathbf{c}^{t-1} \odot \mathbf{f}^t$
*(cell state)*

$\bar{\mathbf{o}}^t = \mathbf{x}^t \mathbf{W}_o + \mathbf{y}^{t-1} \mathbf{R}_o + \mathbf{p}_o \odot \mathbf{c}^{t} + \mathbf{b}_o$<br>
$\mathbf{o}^t = \sigma(\bar{\mathbf{o}}^t)$
*(output gate)*

$\mathbf{y}^t = h(\mathbf{c}^t) \odot \mathbf{o}^t$
*(block output)*


In [4]:
def forward(x, weights):
    Wz, Wi, Wf, Wo, Rz, Ri, Rf, Ro, pi, pf, po, bz, bi, bf, bo = weights
    N = Rz.shape[0]  # nr hidden units
    T, M = x.shape
    za = np.zeros([T, N])
    z  = np.zeros([T, N])
    ia = np.zeros([T, N])
    i  = np.zeros([T, N])
    fa = np.zeros([T, N])
    f  = np.zeros([T, N])
    c  = np.zeros([T, N])
    oa = np.zeros([T, N])
    o  = np.zeros([T, N])
    y  = np.zeros([T, N])
    
    # the t-1 indexing will automatically wrap and access the last timestep which is zero
    for t in range(T):
        za[t] = np.dot(x[t], Wz) + np.dot(y[t-1], Rz) + bz
        z[t] = g(za[t])

        ia[t] = np.dot(x[t], Wi) + np.dot(y[t-1], Ri) + pi*c[t-1] + bi
        i[t] = sigma(ia[t])

        fa[t] = np.dot(x[t], Wf) + np.dot(y[t-1], Rf) + pf*c[t-1] + bf
        f[t] = sigma(fa[t])

        c[t] = i[t] * z[t] + f[t] * c[t-1]

        oa[t] = np.dot(x[t], Wo) + np.dot(y[t-1], Ro) + po*c[t] + bo
        o[t] = sigma(oa[t])

        y[t] = o[t] * h(c[t])
        
    fwd_state = (za, z, ia, i, fa, f, oa, o, c, y)
    return y, fwd_state

## Backward Pass
Let $\Delta^t$ be the vector of deltas received from above. Formally they are $\frac{\partial E}{\partial \mathbf{y}^t}$ but not including the recurrent dependencies. We'll resolve those in $\mathbf{\delta y}^t$:


$\mathbf{\delta y}^t = \Delta^t + \mathbf{\delta z}^{t+1} \mathbf{R}_z^T + 
                                  \mathbf{\delta i}^{t+1} \mathbf{R}_i^T + 
                                  \mathbf{\delta f}^{t+1} \mathbf{R}_f^T + 
                                  \mathbf{\delta o}^{t+1} \mathbf{R}_o^T$

$\mathbf{\delta o}^t = \mathbf{\delta y}^t \odot h(\mathbf{c}^t) \odot \sigma'(\bar{\mathbf{o}}^t) $

$\mathbf{\delta c}^t = \mathbf{\delta y}^t \odot \mathbf{o}^t \odot h'(\mathbf{c}^t) + 
                       \mathbf{p}_o \odot \mathbf{\delta o}^t +
                       \mathbf{p}_i \odot \mathbf{\delta i}^{t+1} +
                       \mathbf{p}_f \odot \mathbf{\delta f}^{t+1} +
                       \mathbf{\delta c}^{t+1} \odot \mathbf{f}^{t+1}$ 
                       
$\mathbf{\delta f}^t = \mathbf{\delta c}^t \odot \mathbf{c}^{t-1} \odot \sigma'(\bar{\mathbf{f}}^t) $

$\mathbf{\delta i}^t = \mathbf{\delta c}^t \odot \mathbf{z}^{t} \odot \sigma'(\bar{\mathbf{i}}^t) $

$\mathbf{\delta z}^t = \mathbf{\delta c}^t \odot \mathbf{i}^{t} \odot g'(\bar{\mathbf{z}}^t) $


Deltas for the inputs. Only needed if there is a layer below that needs training:<br>
$\mathbf{\delta x}^t = \mathbf{\delta z}^t \mathbf{W}_z^T + 
                       \mathbf{\delta i}^t \mathbf{W}_i^T + 
                       \mathbf{\delta f}^t \mathbf{W}_f^T + 
                       \mathbf{\delta o}^t \mathbf{W}_o^T$

Gradients for the weights:<br>
$\delta\mathbf{W}_\star = \sum^T_{t=0} \langle \mathbf{\delta\star}^t, \mathbf{x}^t \rangle$

$\delta\mathbf{R}_\star = \sum^{T-1}_{t=0} \langle \mathbf{\delta\star}^{t+1}, \mathbf{y}^t \rangle$

$\delta\mathbf{p}_i = \sum^{T-1}_{t=0}  \mathbf{c}^t \odot \mathbf{\delta i}^{t+1}$<br>
$\delta\mathbf{p}_f = \sum^{T-1}_{t=0}  \mathbf{c}^t \odot \mathbf{\delta f}^{t+1}$<br>
$\delta\mathbf{p}_o = \sum^{T}_{t=0}    \mathbf{c}^t \odot \mathbf{\delta o}^{t}$

$\delta\mathbf{b}_\star = \sum^{T}_{t=0} \mathbf{\delta\star}^{t}$


In [5]:
def backward(deltas, weights, fwd_state):
    Wz, Wi, Wf, Wo, Rz, Ri, Rf, Ro, pi, pf, po, bz, bi, bf, bo = weights
    za, z, ia, i, fa, f, oa, o, c, y = fwd_state
    T, N = deltas.shape
    M = Wz.shape[0]
    
    # we make the derivative arrays longer so t+1 is automatically zero at the ends
    dz = np.zeros([T+1, N])
    di = np.zeros([T+1, N])
    df = np.zeros([T+1, N])
    dc = np.zeros([T+1, N])
    do = np.zeros([T+1, N])
    dy = np.zeros([T, N])
    dx = np.zeros([T, M])
    
    # initialize gradients
    gradients = [np.zeros_like(w) for w in weights]
    dWz, dWi, dWf, dWo, dRz, dRi, dRf, dRo, dpi, dpf, dpo, dbz, dbi, dbf, dbo = gradients
    
    for t in reversed(range(T)):
        dy[t] = deltas[t] + np.dot(di[t+1], Ri.T) + np.dot(df[t+1], Rf.T)  +\
                            np.dot(do[t+1], Ro.T) + np.dot(dz[t+1], Rz.T)
        do[t] = dy[t] * h(c[t]) * sigma_deriv(oa[t])
        dc[t] = dy[t] * o[t] * h_deriv(c[t]) + po * do[t]
        if t < T-1:
            dc[t] += pi * di[t+1] + pf * df[t+1] + dc[t+1] * f[t+1]
        if t > 0:
            df[t] = dc[t] * c[t-1] * sigma_deriv(fa[t])
        di[t] = dc[t] * z[t] * sigma_deriv(ia[t])
        dz[t] = dc[t] * i[t] * g_deriv(za[t])
        
        # Input Deltas
        dx[t] = np.dot(dz[t], Wz.T) + np.dot(di[t], Wi.T) + np.dot(df[t], Wf.T) + np.dot(do[t], Wo.T)
        
        # Gradients for the weights
        dWz += np.outer(x[t], dz[t])
        dWi += np.outer(x[t], di[t])
        dWf += np.outer(x[t], df[t])
        dWo += np.outer(x[t], do[t])
        dRz += np.outer(y[t], dz[t+1])
        dRi += np.outer(y[t], di[t+1])
        dRf += np.outer(y[t], df[t+1])
        dRo += np.outer(y[t], do[t+1])        
        dpi += c[t] * di[t+1]
        dpf += c[t] * df[t+1]
        dpo += c[t] * do[t]
        dbz += dz[t]
        dbi += di[t]
        dbf += df[t]
        dbo += do[t]

    bwd_state = [dz, di, df, dc, do, dy]
    return dx, gradients, bwd_state

# Finite Differences Checking 
Let's check the gradients using a numerical approximation, to make sure we didn't do any mistakes.

In [6]:
# As loss function we use Squared Error
SE = lambda y, t: 0.5 * np.sum((y-t)**2)
SE_deriv = lambda y, t: y - t

In [7]:
def finite_diff(f, initial_x, eps=1e-10):
    err1 = f(initial_x)
    delta = np.zeros_like(initial_x)
    diff = np.zeros_like(initial_x)
    for i in range(delta.size):
        delta.flat[i] = eps
        err2 = f(initial_x + delta)
        diff.flat[i] = (err2 - err1) / eps
        delta.flat[i] = 0
    return diff

In [8]:
T = 10  # nr_timesteps
x = rnd.randn(T, M)
targets = rnd.randn(T, N)

In [9]:
y, fwd_state = forward(x, weights)
deltas = SE_deriv(y, targets)
dx, gradients, bwd_state = backward(deltas, weights, fwd_state)

In [10]:
def func1(inputs):
    return SE(forward(inputs, weights)[0], targets)

In [11]:
dx_approx = finite_diff(func1, x)

In [12]:
SE(dx, dx_approx)

4.4914192362054091e-09

To make checking the weight-gradients easier, we will place all the weights in one large array. 
So every array in the weights list is actually a slice of the larger array. 

In [13]:
weight_sizes = [w.size for w in weights]
total_nr_weights = sum(weight_sizes)
split_idx = np.hstack(([0], np.cumsum(weight_sizes)))
shapes = [w.shape for w in weights]

In [14]:
def split_weights(all_weights):
    return [all_weights[i:j].reshape(s) for i, j, s in zip(split_idx[:-1], split_idx[1:], shapes)]

In [15]:
def func2(allweights):
    weights = split_weights(allweights)
    return SE(forward(x, weights)[0], targets)

In [16]:
all_weights = rnd.randn(total_nr_weights)
weights = split_weights(all_weights)

In [17]:
grad_approx = finite_diff(func2, all_weights)

In [18]:
y, fwd_state = forward(x, weights )
deltas = y - targets
dx, gradients, bwd_state = backward(deltas, weights, fwd_state)

In [19]:
weight_names = ['Wz', 'Wi', 'Wf', 'Wo', 'Rz', 'Ri', 'Rf', 'Ro', 'pi', 'pf', 'po', 'bz', 'bi', 'bf', 'bo']
total_err = 0

for g_calc, g_approx, name in zip(gradients, split_weights(grad_approx), weight_names):
    print(name, '=', SE(g_calc, g_approx))
    

Wz = 1.6018560246e-09
Wi = 8.05556588901e-10
Wf = 1.98769681238e-09
Wo = 1.18538571315e-09
Rz = 4.39003824181e-09
Ri = 2.24740426348e-09
Rf = 2.8298336655e-09
Ro = 2.09207265237e-09
pi = 6.86448309643e-10
pf = 1.06052334981e-10
po = 5.34335201749e-10
bz = 4.52704935075e-10
bi = 3.14392932658e-10
bf = 1.20972654246e-10
bo = 1.33090539717e-10
